In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from sodapy import Socrata # socrata api python wrapper

/Users/joel5174/miniconda3/envs/la-covid/lib/python3.8/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=os.getenv('ESRI_GIS_PASSWORD')
)

gis

GIS @ https://geoai.maps.arcgis.com version:8.1

# Access LA City Socrata

In [3]:
sc= Socrata('data.lacity.org', None)
sc

In [4]:
sd_lst = sc.datasets()

In [5]:
sd_lst[0]

{'resource': {'name': 'Building and Safety Permit Information',
  'id': 'yv23-pmwf',
  'parent_fxf': [],
  'description': 'The Department of Building and Safety issues permits for the construction, remodeling, and repair of buildings and structures in the City of Los Angeles.  Permits are categorized into building permits, electrical permits, and mechanical permits (which include plumbing, HVAC systems, fire sprinklers, elevators, and pressure vessels).  Depending on the complexity of a project, a permit may be issued the same day with Express Permit or e-Permit ("No Plan Check" category), or a permit may require that the plans be reviewed ("Plan Check" category) by a Building and Safety Plan Check personnel.',
  'attribution': 'LADBS',
  'attribution_link': 'https://creativecommons.org/licenses/by/4.0/',
  'contact_email': 'jing.reyes@lacity.org',
  'type': 'dataset',
  'updatedAt': '2020-05-25T09:17:16.000Z',
  'createdAt': '2014-03-19T14:48:47.000Z',
  'metadata_updated_at': '2019-0

In [6]:
def vals_from_v(v):
    return  v['resource']['id'], v['resource']['name'], v['resource']['description'], v['classification']['categories'], v['classification']['domain_tags'], v['permalink'], v['link'], v['resource']['page_views']['page_views_total']
vals_lst = [vals_from_v(v) for v in sd_lst]
vals_df = pd.DataFrame(vals_lst, columns=['id', 'name', 'description', 'classification', 'domain_tags', 'permalink', 'link', 'page_views'])
vals_df.head()

,id,name,description,classification,domain_tags,permalink,link,page_views
0,yv23-pmwf,Building and Safety Permit Information,The Department of Building and Safety issues p...,"[economy, housing & development, infrastructure]","[permit, construction, construction services, ...",https://data.lacity.org/d/yv23-pmwf,https://data.lacity.org/A-Prosperous-City/Buil...,507370
1,6rrh-rzua,Listing of Active Businesses,Listing of all active businesses currently reg...,[economy],[],https://data.lacity.org/d/6rrh-rzua,https://data.lacity.org/A-Prosperous-City/List...,319068
2,fdwe-pgcu,2014 Registered Foreclosure Properties,The 2014 Foreclosure Properties registered at ...,"[economy, finance]","[hcidla, 2014, foreclosure]",https://data.lacity.org/d/fdwe-pgcu,https://data.lacity.org/A-Well-Run-City/2014-R...,114242
3,qtax-byj7,BAVN Open Bid Opportunities,A listing of open bid opportunities provided b...,"[infrastructure, economy]","[request for interest, request for qualificati...",https://data.lacity.org/d/qtax-byj7,https://data.lacity.org/A-Well-Run-City/BAVN-O...,97880
4,vugs-2e9n,Immigration Workshops Event Calendar,Immigration Workshops around the City of Los A...,"[infrastructure, demographics]","[driver, citizenship, license, drivers, ab60, ...",https://data.lacity.org/d/vugs-2e9n,https://data.lacity.org/A-Well-Run-City/Immigr...,88706


In [7]:
econ_df = vals_df[vals_df.classification.apply(lambda class_lst: 'economy' in class_lst)].copy()
econ_df.head()

,id,name,description,classification,domain_tags,permalink,link,page_views
0,yv23-pmwf,Building and Safety Permit Information,The Department of Building and Safety issues p...,"[economy, housing & development, infrastructure]","[permit, construction, construction services, ...",https://data.lacity.org/d/yv23-pmwf,https://data.lacity.org/A-Prosperous-City/Buil...,507370
1,6rrh-rzua,Listing of Active Businesses,Listing of all active businesses currently reg...,[economy],[],https://data.lacity.org/d/6rrh-rzua,https://data.lacity.org/A-Prosperous-City/List...,319068
2,fdwe-pgcu,2014 Registered Foreclosure Properties,The 2014 Foreclosure Properties registered at ...,"[economy, finance]","[hcidla, 2014, foreclosure]",https://data.lacity.org/d/fdwe-pgcu,https://data.lacity.org/A-Well-Run-City/2014-R...,114242
3,qtax-byj7,BAVN Open Bid Opportunities,A listing of open bid opportunities provided b...,"[infrastructure, economy]","[request for interest, request for qualificati...",https://data.lacity.org/d/qtax-byj7,https://data.lacity.org/A-Well-Run-City/BAVN-O...,97880
6,8cf2-izs5,New building permit,Permit information,"[housing & development, infrastructure, economy]","[plan check, valuation, apartment, 1 or 2 fami...",https://data.lacity.org/d/8cf2-izs5,https://data.lacity.org/A-Prosperous-City/New-...,47986


# Get Business Data

In [8]:
biz_row = econ_df.iloc[1]
biz_row

id                                                        6rrh-rzua
name                                   Listing of Active Businesses
description       Listing of all active businesses currently reg...
classification                                            [economy]
domain_tags                                                      []
permalink                       https://data.lacity.org/d/6rrh-rzua
link              https://data.lacity.org/A-Prosperous-City/List...
page_views                                                   319068
Name: 1, dtype: object

In [9]:
biz_json = sc.get(biz_row.id)
biz_json

[{'location_account': '0000000108-0001-3',
  'business_name': 'PALACE OF VENICE GUEST HOME /C',
  'street_address': '1727 CRENSHAW BLVD',
  'city': 'LOS ANGELES',
  'zip_code': '90019-6037',
  'location_description': '1727 CRENSHAW 90019-6037',
  'naics': '721310',
  'primary_naics_description': 'Rooming & boarding houses',
  'council_district': '10',
  'location_start_date': '1991-05-15T00:00:00.000',
  'location_1': {'latitude': '34.0425', 'longitude': '-118.3295'},
  ':@computed_region_qz3q_ghft': '23080',
  ':@computed_region_kqwf_mjcx': '12',
  ':@computed_region_k96s_3jcv': '648',
  ':@computed_region_tatf_ua23': '1105',
  ':@computed_region_2dna_qi2s': '19'},
 {'location_account': '0000000115-0001-3',
  'business_name': 'VINCENZO LABELLA',
  'street_address': '521 SWARTHMORE AVENUE',
  'city': 'PACIFIC PALISADES',
  'zip_code': '90272-4350',
  'location_description': '521 SWARTHMORE 90272-4350',
  'mailing_address': '521 SWARTHMORE AVENUE',
  'mailing_city': 'PACIFIC PALISADES',

In [10]:
biz_df = pd.DataFrame(biz_json)
biz_df.head()

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_qz3q_ghft,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name
0,0000000108-0001-3,PALACE OF VENICE GUEST HOME /C,1727 CRENSHAW BLVD,LOS ANGELES,90019-6037,1727 CRENSHAW 90019-6037,721310,Rooming & boarding houses,10,1991-05-15T00:00:00.000,...,23080,12,648,1105,19,NaN,NaN,NaN,NaN,NaN
1,0000000115-0001-3,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,521 SWARTHMORE 90272-4350,561500,Travel arrangement & reservation services,11,1990-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,NaN,NaN
2,0000000121-0001-9,WILCARE ECONOMIC DEVELOPMENT CORPORATION,9911 AVALON BLVD,LOS ANGELES,90003-4805,9911 AVALON 90003-4805,721310,Rooming & boarding houses,8,1999-01-01T00:00:00.000,...,22351,14,806,1176,45,448 E 99TH STREET,LOS ANGELES,90003-4804,7,NaN
3,0000000132-0001-7,CARLOS ANGEL,1221 W 7TH STREET SUITE #N-407,LOS ANGELES,90017-2689,1221 7TH 90017-2689,561300,Employment services,1,1999-07-01T00:00:00.000,...,23078,11,564,1378,76,NaN,NaN,NaN,54,NaN
4,0000000133-0001-1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,4917 S BROADWAY,LOS ANGELES,90037-3211,4917 BROADWAY 90037-3211,611000,"Educational services (including schools, colle...",9,1991-01-01T00:00:00.000,...,23668,13,737,655,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,7,NaN


In [11]:
biz_df.iloc[0]

location_account                                               0000000108-0001-3
business_name                                     PALACE OF VENICE GUEST HOME /C
street_address                                                1727 CRENSHAW BLVD
city                                                                 LOS ANGELES
zip_code                                                              90019-6037
location_description                                    1727 CRENSHAW 90019-6037
naics                                                                     721310
primary_naics_description                              Rooming & boarding houses
council_district                                                              10
location_start_date                                      1991-05-15T00:00:00.000
location_1                     {'latitude': '34.0425', 'longitude': '-118.3295'}
:@computed_region_qz3q_ghft                                                23080
:@computed_region_kqwf_mjcx 

# Locate Businesses

In [12]:
addr_filter = (biz_df.location_1.isnull()) & (biz_df.street_address.notnull())

biz_df[addr_filter].head()

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_qz3q_ghft,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name
1,0000000115-0001-3,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,521 SWARTHMORE 90272-4350,561500,Travel arrangement & reservation services,11,1990-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,NaN,NaN
7,0000000150-0001-5,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,2015 W TEMPLE STREET,LOS ANGELES,90026-4913,2015 TEMPLE 90026-4913,611000,"Educational services (including schools, colle...",13,1991-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,NaN,NaN
8,0000000156-0001-2,SPRINGBOARD NON-PROFIT CONSUMER CREDIT MANAGEMENT,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828,1605 OLYMPIC 90015-3828,523900,Other financial investment activities (includi...,1,1999-02-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MONEY MANAGEMENT INTERNATIONAL
20,0000000223-0001-6,HIGHLAND PARK GUEST HOME INC,345 N AVENUE 57,LOS ANGELES,90042-3403,345 AVENUE 57 90042-3403,721310,Rooming & boarding houses,1,1973-03-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,POST OFFICE BOX #50950,LOS ANGELES,90042-,NaN,HIGHLAND PARK GST. HM INC | HIGHLAND PARK GUES...
22,0000000224-0001-1,"ARROWHEAD BRASS PRODUCTS,LLC",5147 ALHAMBRA AVENUE,LOS ANGELES,90032-3413,5147 ALHAMBRA 90032-3413,452000,General merchandise stores,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
biz_df.iloc[7]

location_account                                               0000000150-0001-5
business_name                      A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C
street_address                                              2015 W TEMPLE STREET
city                                                                 LOS ANGELES
zip_code                                                              90026-4913
location_description                                      2015 TEMPLE 90026-4913
naics                                                                     611000
primary_naics_description      Educational services (including schools, colle...
council_district                                                              13
location_start_date                                      1991-01-01T00:00:00.000
location_1                                                                   NaN
:@computed_region_qz3q_ghft                                                  NaN
:@computed_region_kqwf_mjcx 

In [13]:
loc_filter = biz_df.location_1.notnull()

biz_df.loc[loc_filter].head()

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_qz3q_ghft,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name
0,0000000108-0001-3,PALACE OF VENICE GUEST HOME /C,1727 CRENSHAW BLVD,LOS ANGELES,90019-6037,1727 CRENSHAW 90019-6037,721310,Rooming & boarding houses,10,1991-05-15T00:00:00.000,...,23080,12,648,1105,19,NaN,NaN,NaN,NaN,NaN
2,0000000121-0001-9,WILCARE ECONOMIC DEVELOPMENT CORPORATION,9911 AVALON BLVD,LOS ANGELES,90003-4805,9911 AVALON 90003-4805,721310,Rooming & boarding houses,8,1999-01-01T00:00:00.000,...,22351,14,806,1176,45,448 E 99TH STREET,LOS ANGELES,90003-4804,7,NaN
3,0000000132-0001-7,CARLOS ANGEL,1221 W 7TH STREET SUITE #N-407,LOS ANGELES,90017-2689,1221 7TH 90017-2689,561300,Employment services,1,1999-07-01T00:00:00.000,...,23078,11,564,1378,76,NaN,NaN,NaN,54,NaN
4,0000000133-0001-1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,4917 S BROADWAY,LOS ANGELES,90037-3211,4917 BROADWAY 90037-3211,611000,"Educational services (including schools, colle...",9,1991-01-01T00:00:00.000,...,23668,13,737,655,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,7,NaN
5,0000000141-0001-6,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,1330 WILSHIRE BLVD #208,LOS ANGELES,90017-1705,1330 WILSHIRE 90017-1705,611000,"Educational services (including schools, colle...",1,1991-01-01T00:00:00.000,...,23078,11,564,1047,68,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,54,NaN


In [14]:
def loc_to_geom(coords):
    return Geometry({'x': float(coords['longitude']), 'y': float(coords['latitude']), 'spatialReference': {'wkid': 4326}})

biz_df.loc[loc_filter, 'SHAPE'] = biz_df[loc_filter].location_1.apply(lambda coords: loc_to_geom(coords))

biz_df.spatial.set_geometry('SHAPE')

biz_df.head()

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name,SHAPE
0,0000000108-0001-3,PALACE OF VENICE GUEST HOME /C,1727 CRENSHAW BLVD,LOS ANGELES,90019-6037,1727 CRENSHAW 90019-6037,721310,Rooming & boarding houses,10,1991-05-15T00:00:00.000,...,12,648,1105,19,NaN,NaN,NaN,NaN,NaN,"{""x"": -118.3295, ""y"": 34.0425, ""spatialReferen..."
1,0000000115-0001-3,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,521 SWARTHMORE 90272-4350,561500,Travel arrangement & reservation services,11,1990-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,NaN,NaN,None
2,0000000121-0001-9,WILCARE ECONOMIC DEVELOPMENT CORPORATION,9911 AVALON BLVD,LOS ANGELES,90003-4805,9911 AVALON 90003-4805,721310,Rooming & boarding houses,8,1999-01-01T00:00:00.000,...,14,806,1176,45,448 E 99TH STREET,LOS ANGELES,90003-4804,7,NaN,"{""x"": -118.2651, ""y"": 33.9463, ""spatialReferen..."
3,0000000132-0001-7,CARLOS ANGEL,1221 W 7TH STREET SUITE #N-407,LOS ANGELES,90017-2689,1221 7TH 90017-2689,561300,Employment services,1,1999-07-01T00:00:00.000,...,11,564,1378,76,NaN,NaN,NaN,54,NaN,"{""x"": -118.2665, ""y"": 34.0518, ""spatialReferen..."
4,0000000133-0001-1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,4917 S BROADWAY,LOS ANGELES,90037-3211,4917 BROADWAY 90037-3211,611000,"Educational services (including schools, colle...",9,1991-01-01T00:00:00.000,...,13,737,655,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,7,NaN,"{""x"": -118.2783, ""y"": 33.9981, ""spatialReferen..."


In [15]:
biz_df.spatial.plot(gis=GIS())

/Users/joel5174/miniconda3/envs/la-covid/lib/python3.8/site-packages/arcgis/features/geo/_accessor.py:2362: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime)):


MapView(layout=Layout(height='400px', width='100%'))

# Identify _Null Island_ and Unlocated Data

In [22]:
biz_df.iloc[422]

location_account                                               0000001436-0233-8
business_name                                                SP PLUS CORPORATION
street_address                                                  110 E 9TH STREET
city                                                                 LOS ANGELES
zip_code                                                              90079-1300
location_description                                          110 9TH 90079-1300
naics                                                                     812930
primary_naics_description                                 Parking lots & garages
council_district                                                              14
location_start_date                                      2009-06-01T00:00:00.000
location_1                               {'latitude': '0.0', 'longitude': '0.0'}
:@computed_region_qz3q_ghft                                                  NaN
:@computed_region_kqwf_mjcx 

In [24]:
zero_filter = biz_df.SHAPE.apply(lambda geom: geom.centroid == (0, 0) if geom else False)

biz_df.loc[zero_filter]

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name,SHAPE
242,0000001051-0001-9,SPECIAL SERVICE FOR GROUPS,15250 SHERMAN WAY UNIT #200,VAN NUYS,-,15250 SHERMAN WAY -,621399,Offices of all other miscellaneous health prac...,6,1993-09-29T00:00:00.000,...,NaN,NaN,NaN,NaN,905 E 8TH STREET,LOS ANGELES,90021-1848,NaN,ASIAN PACIFIC COUNSELING & TREATMENT CENTERS,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
331,0000001305-0001-3,JEREMY P HOWARD,14320 VENTURA BLVD SUITE #502,SHERMAN OAKS,91423-2717,14320 VENTURA 91423-2717,512100,Motion picture & video industries (except vide...,4,1999-02-02T00:00:00.000,...,NaN,NaN,NaN,NaN,14320 VENTURA BLVD SUITE #502,SHERMAN OAKS,91423-2717,NaN,NaN,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
422,0000001436-0233-8,SP PLUS CORPORATION,110 E 9TH STREET,LOS ANGELES,90079-1300,110 9TH 90079-1300,812930,Parking lots & garages,14,2009-06-01T00:00:00.000,...,NaN,NaN,NaN,NaN,3470 WILSHIRE BLVD SUITE #400,LOS ANGELES,90010-2207,NaN,LOC 81787,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
448,0000001436-0310-5,SP PLUS CORPORATION,428 E TEMPLE STREET,LOS ANGELES,90012-0000,428 TEMPLE 90012-0000,812930,Parking lots & garages,14,2016-05-23T00:00:00.000,...,NaN,NaN,NaN,NaN,3470 WILSHIRE BLVD SUITE #400,LOS ANGELES,90010-3927,NaN,SP PLUS #82607,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
890,0000003440-0001-8,FRED POSTAL,110 E 9TH STREET SUITE #A301,LOS ANGELES,90079-1301,110 9TH 90079-1301,424300,"Apparel, piece goods, & notions",14,1992-02-14T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRED POSTAL ASSOC,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
966,0000003961-0008-8,MORRIS YOUNG,4353 DON MIGUEL DRIVE,LOS ANGELES,90008-,4353 DON MIGUEL 90008-,531100,Lessors of real estate (including mini warehou...,8,2007-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,POST OFFICE BOX #56274,LOS ANGELES,90056-0274,NaN,NaN,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
967,0000003961-0009-6,MORRIS YOUNG,4243 4255 LEIMERT BLVD,LOS ANGELES,90008-4605,4243 LEIMERT 90008-4605,531100,Lessors of real estate (including mini warehou...,0,2007-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,POST OFFICE BOX #56274,LOS ANGELES,90056-0274,NaN,NaN,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
980,0000004043-0001-6,ALBEA BEAUTY SOLUTIONS USA LLC,595 MADISON AVENUE,NEW YORK,10022-1907,595 MADISON 10022-1907,424600,Chemical & allied products,0,1991-12-01T00:00:00.000,...,NaN,NaN,NaN,NaN,595 MADISON AVENUE,NEW YORK,10022-1907,NaN,"ALBEA BEAUTY SOLUTION USA, LLC","{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."


In [25]:
gcd_filter = addr_filter | zero_filter

biz_df[gcd_filter]

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name,SHAPE
1,0000000115-0001-3,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,521 SWARTHMORE 90272-4350,561500,Travel arrangement & reservation services,11,1990-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,NaN,NaN,None
7,0000000150-0001-5,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,2015 W TEMPLE STREET,LOS ANGELES,90026-4913,2015 TEMPLE 90026-4913,611000,"Educational services (including schools, colle...",13,1991-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,NaN,NaN,None
8,0000000156-0001-2,SPRINGBOARD NON-PROFIT CONSUMER CREDIT MANAGEMENT,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828,1605 OLYMPIC 90015-3828,523900,Other financial investment activities (includi...,1,1999-02-01T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MONEY MANAGEMENT INTERNATIONAL,None
20,0000000223-0001-6,HIGHLAND PARK GUEST HOME INC,345 N AVENUE 57,LOS ANGELES,90042-3403,345 AVENUE 57 90042-3403,721310,Rooming & boarding houses,1,1973-03-01T00:00:00.000,...,NaN,NaN,NaN,NaN,POST OFFICE BOX #50950,LOS ANGELES,90042-,NaN,HIGHLAND PARK GST. HM INC | HIGHLAND PARK GUES...,None
22,0000000224-0001-1,"ARROWHEAD BRASS PRODUCTS,LLC",5147 ALHAMBRA AVENUE,LOS ANGELES,90032-3413,5147 ALHAMBRA 90032-3413,452000,General merchandise stores,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,0000003646-0001-1,"ROYALE GRACE INVESTMENT COMPANY, LLC",8720 BURTON WAY,LOS ANGELES,90048-3810,8720 BURTON 90048-3810,531100,Lessors of real estate (including mini warehou...,5,1998-12-24T00:00:00.000,...,NaN,NaN,NaN,NaN,10660 WHITE OAK AVENUE,GRANADA HILLS,91344-5943,NaN,NaN,None
940,0000003830-0001-7,MARUBINI AMERICA CORP,6075 POPLAR AVENUE #500,MEMPHIS,38119-,6075 POPLAR 38119,115310,Support activities for forestry,0,1992-02-03T00:00:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
966,0000003961-0008-8,MORRIS YOUNG,4353 DON MIGUEL DRIVE,LOS ANGELES,90008-,4353 DON MIGUEL 90008-,531100,Lessors of real estate (including mini warehou...,8,2007-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,POST OFFICE BOX #56274,LOS ANGELES,90056-0274,NaN,NaN,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."
967,0000003961-0009-6,MORRIS YOUNG,4243 4255 LEIMERT BLVD,LOS ANGELES,90008-4605,4243 LEIMERT 90008-4605,531100,Lessors of real estate (including mini warehou...,0,2007-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,POST OFFICE BOX #56274,LOS ANGELES,90056-0274,NaN,NaN,"{""x"": 0.0, ""y"": 0.0, ""spatialReference"": {""wki..."


# Geocode [Null Island](https://en.wikipedia.org/wiki/Null_Island) and Unlocated Data

- https://developers.arcgis.com/python/guide/batch-geocoding/
- https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html#batch-geocode

In [26]:
from arcgis import geocoding

In [37]:
gcd_df = biz_df[gcd_filter][['street_address', 'city', 'zip_code']].copy()
gcd_df.columns = ['Address', 'City', 'Postal']
gcd_df.head()

,Address,City,Postal
1,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350
7,2015 W TEMPLE STREET,LOS ANGELES,90026-4913
8,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828
20,345 N AVENUE 57,LOS ANGELES,90042-3403
22,5147 ALHAMBRA AVENUE,LOS ANGELES,90032-3413


In [43]:
gcd_arr = eval(gcd_df.to_json(orient='records'))
gcd_arr

[{'Address': '521 SWARTHMORE AVENUE',
  'City': 'PACIFIC PALISADES',
  'Postal': '90272-4350'},
 {'Address': '2015 W TEMPLE STREET',
  'City': 'LOS ANGELES',
  'Postal': '90026-4913'},
 {'Address': '1605 W OLYMPIC BLVD #9023',
  'City': 'LOS ANGELES',
  'Postal': '90015-3828'},
 {'Address': '345   N AVENUE 57',
  'City': 'LOS ANGELES',
  'Postal': '90042-3403'},
 {'Address': '5147 ALHAMBRA AVENUE',
  'City': 'LOS ANGELES',
  'Postal': '90032-3413'},
 {'Address': '1715 W FLORENCE AVENUE',
  'City': 'LOS ANGELES',
  'Postal': '90047-2220'},
 {'Address': '3806 W PICO BLVD',
  'City': 'LOS ANGELES',
  'Postal': '90019-4304'},
 {'Address': '260 S UNION AVENUE #1',
  'City': 'LOS ANGELES',
  'Postal': '90026-5452'},
 {'Address': '3115 VENICE BLVD',
  'City': 'LOS ANGELES',
  'Postal': '90019-6238'},
 {'Address': '2601 W PICO BLVD #2',
  'City': 'LOS ANGELES',
  'Postal': '90006-3901'},
 {'Address': '4151 BEVERLY BLVD',
  'City': 'LOS ANGELES',
  'Postal': '90004-4461'},
 {'Address': '3049 W 

In [40]:
address_locator = geocoding.get_geocoders(gis)[0]
address_locator

<Geocoder url:"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">

In [44]:
gcd_res = geocoding.batch_geocode(gcd_arr, out_sr=4326, geocoder=address_locator)
gcd_res

[{'address': '521 Swarthmore Ave, Pacific Palisades, California, 90272',
  'location': {'x': -118.52812891499997, 'y': 34.03859584300005},
  'score': 100,
  'attributes': {'ResultID': 0,
   'Loc_name': 'World',
   'Status': 'M',
   'Score': 100,
   'Match_addr': '521 Swarthmore Ave, Pacific Palisades, California, 90272',
   'LongLabel': '521 Swarthmore Ave, Pacific Palisades, CA, 90272, USA',
   'ShortLabel': '521 Swarthmore Ave',
   'Addr_type': 'PointAddress',
   'Type': '',
   'PlaceName': '',
   'Place_addr': '521 Swarthmore Ave, Pacific Palisades, California, 90272',
   'Phone': '',
   'URL': '',
   'Rank': 20,
   'AddBldg': '',
   'AddNum': '521',
   'AddNumFrom': '',
   'AddNumTo': '',
   'AddRange': '',
   'Side': '',
   'StPreDir': '',
   'StPreType': '',
   'StName': 'Swarthmore',
   'StType': 'Ave',
   'StDir': '',
   'BldgType': '',
   'BldgName': '',
   'LevelType': '',
   'LevelName': '',
   'UnitType': '',
   'UnitName': '',
   'SubAddr': '',
   'StAddr': '521 Swarthmore

In [45]:
def res_to_geom(mtch):
    loc = mtch['location']
    loc['spatialReference'] = {'wkid': 4326}
    return Geometry(loc)

fix_geom_srs = pd.Series([res_to_geom(mtch) for mtch in gcd_res])
fix_geom_srs

0     {'x': -118.52812891499997, 'y': 34.03859584300...
1     {'x': -118.26683979099994, 'y': 34.06989950900...
2     {'x': -118.27428299999997, 'y': 34.04976300000...
3     {'x': -118.19422350199994, 'y': 34.11316335900...
4     {'x': -118.17368694999999, 'y': 34.07254053100...
                            ...                        
72    {'x': -118.38231659399997, 'y': 34.07205271900...
73    {'x': -89.86084718999996, 'y': 35.102126103000...
74    {'x': -118.35088054999994, 'y': 34.00522380100...
75    {'x': -118.32824555299999, 'y': 34.00668684000...
76    {'x': -73.97216213799999, 'y': 40.762509905000...
Length: 77, dtype: object

In [46]:
gcd_df['SHAPE'] = fix_geom_srs
gcd_df.spatial.set_geometry('SHAPE')
gcd_df.head()

,Address,City,Postal,SHAPE
1,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,"{""x"": -118.26683979099994, ""y"": 34.06989950900..."
7,2015 W TEMPLE STREET,LOS ANGELES,90026-4913,"{""x"": -118.26720959899995, ""y"": 34.06201395000..."
8,1605 W OLYMPIC BLVD #9023,LOS ANGELES,90015-3828,"{""x"": -118.318408853, ""y"": 34.04359925600005, ..."
20,345 N AVENUE 57,LOS ANGELES,90042-3403,"{""x"": -118.33849153999995, ""y"": 34.00619210100..."
22,5147 ALHAMBRA AVENUE,LOS ANGELES,90032-3413,"{""x"": -118.3106115, ""y"": 33.93087551700006, ""s..."


In [47]:
gcd_df.spatial.plot(gis=GIS())

MapView(layout=Layout(height='400px', width='100%'))

# Add Locations Back Into Master Data

In [48]:
biz_df.loc[gcd_filter, 'SHAPE'] = gcd_df.SHAPE

In [49]:
biz_df.spatial.plot(gis=GIS())

/Users/joel5174/miniconda3/envs/la-covid/lib/python3.8/site-packages/arcgis/features/geo/_accessor.py:2362: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime)):


MapView(layout=Layout(height='400px', width='100%'))

In [50]:
biz_df.head()

,location_account,business_name,street_address,city,zip_code,location_description,naics,primary_naics_description,council_district,location_start_date,...,:@computed_region_kqwf_mjcx,:@computed_region_k96s_3jcv,:@computed_region_tatf_ua23,:@computed_region_2dna_qi2s,mailing_address,mailing_city,mailing_zip_code,:@computed_region_ur2y_g4cx,dba_name,SHAPE
0,0000000108-0001-3,PALACE OF VENICE GUEST HOME /C,1727 CRENSHAW BLVD,LOS ANGELES,90019-6037,1727 CRENSHAW 90019-6037,721310,Rooming & boarding houses,10,1991-05-15T00:00:00.000,...,12,648,1105,19,NaN,NaN,NaN,NaN,NaN,"{""x"": -118.3295, ""y"": 34.0425, ""spatialReferen..."
1,0000000115-0001-3,VINCENZO LABELLA,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,521 SWARTHMORE 90272-4350,561500,Travel arrangement & reservation services,11,1990-01-01T00:00:00.000,...,NaN,NaN,NaN,NaN,521 SWARTHMORE AVENUE,PACIFIC PALISADES,90272-4350,NaN,NaN,"{""x"": -118.26683979099994, ""y"": 34.06989950900..."
2,0000000121-0001-9,WILCARE ECONOMIC DEVELOPMENT CORPORATION,9911 AVALON BLVD,LOS ANGELES,90003-4805,9911 AVALON 90003-4805,721310,Rooming & boarding houses,8,1999-01-01T00:00:00.000,...,14,806,1176,45,448 E 99TH STREET,LOS ANGELES,90003-4804,7,NaN,"{""x"": -118.2651, ""y"": 33.9463, ""spatialReferen..."
3,0000000132-0001-7,CARLOS ANGEL,1221 W 7TH STREET SUITE #N-407,LOS ANGELES,90017-2689,1221 7TH 90017-2689,561300,Employment services,1,1999-07-01T00:00:00.000,...,11,564,1378,76,NaN,NaN,NaN,54,NaN,"{""x"": -118.2665, ""y"": 34.0518, ""spatialReferen..."
4,0000000133-0001-1,A A OFICINA CENTRAL HISPANA DE LOS ANGELES /C,4917 S BROADWAY,LOS ANGELES,90037-3211,4917 BROADWAY 90037-3211,611000,"Educational services (including schools, colle...",9,1991-01-01T00:00:00.000,...,13,737,655,NaN,2607 VAN BUREN PLACE,LOS ANGELES,90007-2129,7,NaN,"{""x"": -118.2783, ""y"": 33.9981, ""spatialReferen..."
